# 노인보호구역 map

In [1]:
import requests, json

def get_location(address):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
                      # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
                      # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
        headers = {"Authorization": "KakaoAK aa058a07922433baf688d53c0c06e5ea"}
        api_json = json.loads(str(requests.get(url,headers=headers).text))
        address = api_json['documents'][0]['address']
        rcd =str(address['y']) + ','+ str(address['x'])
        address_name = address['address_name']
        return rcd
        
    
    except:
        return 'r,c'

In [2]:
import pandas as pd
# 노인보호구역
silver_csv = pd.read_csv('silver_zone.csv')
print(silver_csv.tail())

      연번  자치구         시설명             위치   지정일
165  166  강동구  성암장수마을 요양원     구천면로68길 46  2016
166  167  강동구   강동노인종합복지관    동남로71길 32-5  2018
167  168  강동구       쉼터경로당  올림픽로89길 27-23  2019
168  169  강동구     구립강암경로당   상암로15길 38-15  2020
169  170  강동구     일자산자연공원      둔촌동 산 125  2021


In [3]:
#노인보호구역의 주소를 위도 경도로 변환
silver_address= silver_csv['위치']
latitude = []
longitude = []
for i in silver_address:
    lat, lng = get_location(i).split(',')
    latitude.append(lat)
    longitude.append(lng)

In [5]:
silver_address_df = pd.DataFrame({
                            "자치구" : silver_csv["자치구"], 
                            "시설명": silver_csv["시설명"],
                           '주소':silver_csv['위치'],
                            "지정일": silver_csv["지정일"],
                           '위도':latitude,
                           '경도':longitude})

silver_address_df.to_csv("silver_zone_df.csv")

In [6]:
x = []
y = []
for i in range(len(silver_address_df['위도'])):
    if silver_address_df['위도'][i] == 0.0 or silver_address_df['경도'][i] == 0.0:
        pass
    else:
        x.append(silver_address_df['위도'][i])
        y.append(silver_address_df['경도'][i])
print('x갯수: ',len(x))
print('y갯수: ',len(y))

x갯수:  170
y갯수:  170


In [8]:
import folium
map_osm = folium.Map(location=[x[13],y[13]],zoom_start=14)
for i in range(len(x)):
    folium.Marker([x[i],y[i]], popup='<pre>%s</pre>'%silver_address_df.시설명[i], icon=folium.Icon(color='red', icon='info-sign')).add_to(map_osm)
# 지도에서 범위 지정
folium.CircleMarker(location=[x[13],y[13]], popup='서울', radius=300, color="#3186CC", fill_color="#3186CC").add_to(map_osm)
#map저장
map_osm.save('Seoul_trans.html')

In [9]:
map_osm